In [1]:
%matplotlib notebook

import argparse, os, sys
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from chenfeng.data_tools import BaseDataset, paths_for_dataset
from ASEUNet import SEResUNet

def parse_args():
    parser = argparse.ArgumentParser("")
    parser.add_argument("--train_dir")
    # return parser.parse_args()
    # So that this works with jupyter
    return parser.parse_args(args=[
        "--train_dir",
        "/rdfs/fast/home/sunyingge/data/COV_19/prced_0512/Train/"
    ])

args = parse_args()

class Args(object):
    def __init__(self):
        self.gpu = "3"
        self.n_channels = 2
        self.n_classes = 1
        self.epochs = 80
        self.initial_epoch = 0
        self.global_step = 0
        self.batchsize = 64
        self.lr = 0.0005
        self.percentage = 0.1
        self.load = False
        self.save_cp = True
        self.img_size = (256, 256)
        self.description = "HRNET_KERAS_DEFAULT"

        self.steps = [5000, 10000]
        self.lr = [1e-3, 5e-4, 25e-5]

model_args = Args()
os.environ["CUDA_VISIBLE_DEVICES"] = model_args.gpu

print("==>>Training set: ")
train_all, train_pos, train_neg = paths_for_dataset(args.train_dir,
    flags=["train"],
    seed=999,
    isprint=True)

train_paths_cons  = train_pos["train"]
train_paths_chan  = []
np.random.seed(999)
train_paths_cons = np.random.permutation(train_paths_cons).tolist()
train_paths_chan = np.random.permutation(train_paths_chan).tolist()

print("++"*30)
print(f"Length of train_paths_cons: {len(train_paths_cons)}")
print(f"Length of train_paths_chan: {len(train_paths_chan)}")

train_dataset = BaseDataset(train_paths_cons, train_paths_chan, img_size=model_args.img_size, choice="all",
                          image_key="image", mask_key="mask")
print(f"train_dataset: {len(train_dataset)}")

Using TensorFlow backend.


==>>Training set: 
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>train<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
all: 487633
pos: 94361
neg: 393272
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Length of train_paths_cons: 94361
Length of train_paths_chan: 0
train_dataset: 94361


In [2]:
input_im = tf.placeholder(tf.float32, shape=(None, model_args.img_size[0], model_args.img_size[1], 1))
input_ann = tf.placeholder(tf.float32, shape=(None, model_args.img_size[0], model_args.img_size[1], 1))
pred = SEResUNet(input_im, num_classes=1, reduction=8, name_scope="SEResUNet")
ce = tf.nn.softmax_cross_entropy_with_logits(labels=input_ann, logits=pred)
loss = tf.reduce_mean(ce)

global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.piecewise_constant(global_step, model_args.steps, model_args.lr)
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step)

config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.95


Instructions for updating:
Use keras.layers.AveragePooling2D instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.


Instructions for updating:
Use `tf.keras.layers.Conv2DTranspose` instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [3]:
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(model_args.epochs):
        print(f"Epoch {epoch + 1}\n")
        num_batches = len(train_dataset) // model_args.batchsize
        for i in range(num_batches):
            print(f"Epoch progress: {i + 1} / {num_batches}")
            data_list = []
            for j in range(i * model_args.batchsize, (i + 1) * model_args.batchsize):
                data_list.append(train_dataset[j])
            data_ar = np.array(data_list)
            im_ar, ann_ar = data_ar[:,0,:,:,:], data_ar[:,1,:,:,:]

            ret_loss, ret_pred, ret_ce = sess.run([loss, pred, ce],
                feed_dict={
                    input_im: im_ar,
                    input_ann: ann_ar,
                    })
            print(np.sum(ret_ce))

Epoch 1

Epoch progress: 1 / 1474
0.0
Epoch progress: 2 / 1474
0.0
Epoch progress: 3 / 1474
0.0
Epoch progress: 4 / 1474
0.0
Epoch progress: 5 / 1474
0.0
Epoch progress: 6 / 1474
0.0
Epoch progress: 7 / 1474


KeyboardInterrupt: 